# Simple Weather Agent

In this notebook, we will create a simple weather agent that can answer questions about the weather. 

In [1]:
import os
import requests

from openai import OpenAI
from dotenv import load_dotenv

In [2]:
_ = load_dotenv()
openai_key = os.getenv("OPENAI_API_KEY")
weatherstack_key = os.getenv("WEATHERSTACK_API_KEY")

llm_name = "gpt-5-mini"

client = OpenAI(api_key=openai_key)

## Tools for AI Agent

In [22]:
def get_weather_forecast(city_name):
    """
    Retrieves the weather forecast for a given city using the WeatherStack API.

    Args:
        city_name (str): The name of the city.

    Returns:
        dict: The weather forecast data if successful, or an error message.
    """
    base_url = "http://api.weatherstack.com/current"
    params = {
        "query": city_name,
        "access_key": weatherstack_key
    }

    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()  # Raise an HTTPError for bad responses (4xx and 5xx)
        json_response = response.json()
        response = {
            "Temperature" : json_response['current']['temperature'],
            "Conditions" : json_response['current']['weather_descriptions'],
            "Humidity" : json_response['current']['humidity'],
            "Wind Speed" : json_response['current']['wind_speed'],
            "Visibility" : json_response['current']['visibility']
        }
        return response
    except requests.exceptions.RequestException as e:
        return {"error": str(e)}

## AI Agent

In [4]:
class WeatherAgent:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message):
        self.messages.append({"role": "user", "content" : message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content" : result})
        return result

    def execute(self):
        response = client.chat.completions.create(
            model=llm_name,
            messages=self.messages
        )
        return response.choices[0].message.content

## Prompt

In [5]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop, you output an Answer.
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

city_weather:
e.g. city_weather: Mumbai
Fetches the weather through an API call and returns the weather observations

Example session:

Question: What is the forecast for Mumbai today?
Thought: I should find the temperature, weather description, humidity and visibility
Action: city_weather: Mumbai
PAUSE

Observation: The temperature in Mumbai is 27 degrees, there are Rain showers expected. Humidity is at 89% and visibility is about 3 kilometers 

Answer: The temperature in Mumbai is 27 degrees, there are Rain showers expected. Humidity is at 89% and visibility is about 3 kilometers 
""".strip()

In [6]:
known_actions = {"city_weather" : get_weather_forecast}

## Interact with Agent

In [7]:
weatherAgent = WeatherAgent(system=prompt)

In [8]:
weatherAgent("What is the forecast for Mumbai today?")

'Thought: I should find the current temperature, weather description (conditions), humidity, wind, and visibility for Mumbai today.\nAction: city_weather: Mumbai\nPAUSE'

In [23]:
response = get_weather_forecast("Mumbai")

In [24]:
print(response)

{'Temperature': 27, 'Conditions': ['Mist'], 'Humidity': 89, 'Wind Speed': 29, 'Visibility': 3}


In [25]:
next_response = f"Observation: {response}"

In [26]:
print(next_response)

Observation: {'Temperature': 27, 'Conditions': ['Mist'], 'Humidity': 89, 'Wind Speed': 29, 'Visibility': 3}


In [27]:
print(f" Messages--> {weatherAgent.messages}")

 Messages--> [{'role': 'system', 'content': 'You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop, you output an Answer.\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncity_weather:\ne.g. city_weather: Mumbai\nFetches the weather through an API call and returns the weather observations\n\nExample session:\n\nQuestion: What is the forecast for Mumbai today?\nThought: I should find the temperature, weather description, humidity and visibility\nAction: city_weather: Mumbai\nPAUSE\n\nObservation: The temperature in Mumbai is 27 degrees, there are Rain showers expected. Humidity is at 89% and visibility is about 3 kilometers \n\nAnswer: The temperature in Mumbai is 27 degrees, there are Rain showers expected. Humidity is at 89% and visibility is about 3 kilometers'}